In [1]:
import pickle
import numpy as np
import bokeh

In [2]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, BoxZoomTool, ResetTool, LassoSelectTool, CategoricalColorMapper
                                  
from bokeh.io import output_notebook
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

# Saved t-SNE

In [3]:
embed_2d = pickle.load(open('data_visualisation/2d_embeddings.pkl', 'rb'))
idxs, embed = zip(*embed_2d.items())
embed = np.c_[embed]

In [4]:
r_paths = [f'data_visualisation/test_imgs/{idx}_r.svg' for idx in idxs]
p_paths = [f'data_visualisation/test_imgs/{idx}_p.svg' for idx in idxs]

In [5]:
output_file("reactions_tsne.html")

source = ColumnDataSource(
        data=dict(
            x=embed[0],
            y=embed[1],
            r_imgs = r_paths,
            p_imgs = p_paths
        )
    )

hover = HoverTool(
        tooltips="""
            <div class="row">
                  <div class="column">
                      <embed type="image/svg+xml" src="@r_imgs" width="200px" height="200px"/>
                  </div>
                  <div class="column">
                      <embed type="image/svg+xml" src="@p_imgs" width="200px" height="200px"/>
                  </div>
            </div>
            """
    )

p = figure(plot_width=800, plot_height=800, tools=[hover, BoxZoomTool(), ResetTool(), LassoSelectTool()],
           title="Embedding of reactions in 2D space", margin=200)

p.circle('x', 'y', size=5, source=source)

show(p)

# Your own t-SNE

In [6]:
from sklearn.cluster import KMeans
from MulticoreTSNE import MulticoreTSNE as TSNE

In [7]:
embed = pickle.load(open('data_visualisation/original_embeddings.pkl', 'rb'))
idxs, embed = zip(*embed.items())
embed = np.c_[embed].T

In [8]:
embed_2d = TSNE(n_jobs=20).fit_transform(embed)

In [9]:
cls = KMeans(n_clusters=15).fit_predict(embed)

In [10]:
output_file("reactions_own_tsne.html")

source = ColumnDataSource(
        data=dict(
            x=embed_2d[:, 0],
            y=embed_2d[:, 1],
            r_imgs = r_paths,
            p_imgs = p_paths, 
            c = cls.astype(str)
        )
    )

hover = HoverTool(
        tooltips="""
            <div class="row">
                  <div class="column">
                      <embed type="image/svg+xml" src="@r_imgs" width="300px" height="300px"/>
                  </div>
                  <div class="column">
                      <embed type="image/svg+xml" src="@p_imgs" width="300px" height="300px"/>
                  </div>
            </div>
            """
    )
palette = d3['Category20'][15]
color_map = CategoricalColorMapper(factors=np.arange(15).astype(str), palette=palette)

p = figure(plot_width=800, plot_height=800, tools=[hover, BoxZoomTool(), ResetTool(), LassoSelectTool()],
           title="Embedding of reactions in 2D space", margin=200)

p.circle('x', 'y', size=5, source=source, color={'field': 'c', 'transform': color_map},)

show(p)